In [1]:
import pandas as pd

In [3]:
preprocessed_dataset = 'edge_iiot_preprocessed.csv'  # Replace with your actual path
df = pd.read_csv(preprocessed_dataset, low_memory=False)

In [52]:
df.head()

,time_difference,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,http.content_length,http.request.method,http.referer,http.request.version,http.response,...,mqtt.hdrflags,mqtt.len,mqtt.msgtype,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,Attack_label,Attack_type
0,-0.000309,-0.047835,0.0,-0.219227,-0.220062,-0.040216,0.131863,0.235181,-0.005951,0,...,-0.300891,-0.281368,-0.300891,0.0,-0.460693,-0.460680,0.0,0.0,0,Normal
1,0.000818,-0.047835,0.0,-0.219227,-0.220062,-0.040216,0.131863,0.235181,-0.005951,0,...,0.044291,1.180232,0.044291,4.0,3.395124,-0.460680,0.0,4.0,0,Normal
2,-0.000308,-0.047835,0.0,-0.219227,-0.220062,-0.040216,0.131863,0.235181,-0.005951,0,...,-0.300891,-0.281368,-0.300891,0.0,-0.460693,-0.460680,0.0,0.0,0,Normal
3,-0.000304,-0.047835,0.0,-0.219227,-0.220062,-0.040216,0.131863,0.235181,-0.005951,0,...,0.389473,-0.037768,0.389473,0.0,-0.460693,-0.460680,0.0,0.0,0,Normal
4,-0.000260,-0.047835,0.0,-0.219227,-0.220062,-0.040216,0.131863,0.235181,-0.005951,0,...,0.734654,4.468833,0.734654,0.0,-0.460693,3.395354,24.0,0.0,0,Normal


In [37]:
X = df.drop(['Attack_label', 'Attack_type'], axis=1)
y = df['Attack_label']  # for binary classification (0 or 1)


In [45]:
# y
unique_values = df['Attack_label'].unique()

print(unique_values)

[0 1]


#### Reshape

In [38]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert to tensors
X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.long)

# Reshape for Conv1D: (batch_size, in_channels=1, sequence_length)
X_tensor = X_tensor.unsqueeze(1)

#### Create DataLoader

In [39]:
dataset = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

#### Loss and Optimizer

In [50]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [61]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Convert to tensors and reshape: (N, 1, num_features)
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Datasets and DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)


In [63]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleConv1DModel(nn.Module):
    def __init__(self, input_length):
        super(SimpleConv1DModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=8, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)
        conv_out_size = (input_length - 2) // 2  # Adjust based on kernel/pool
        self.fc1 = nn.Linear(8 * conv_out_size, 16)
        self.fc2 = nn.Linear(16, 2)  # Binary classification

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # (batch, 8, conv_out_size)
        x = x.view(x.size(0), -1)             # Flatten
        x = F.relu(self.fc1(x))
        return self.fc2(x)


In [62]:
input_length = X_train.shape[1]  # number of features
model = SimpleConv1DModel(input_length=input_length)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleConv1DModel(nn.Module):
    def __init__(self, input_length):
        super(SimpleConv1DModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=8, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)
        conv_out_size = (input_length - 2) // 2  # Adjust based on kernel/pool
        self.fc1 = nn.Linear(8 * conv_out_size, 16)
        self.fc2 = nn.Linear(16, 2)  # Binary classification

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # (batch, 8, conv_out_size)
        x = x.view(x.size(0), -1)             # Flatten
        x = F.relu(self.fc1(x))
        return self.fc2(x)


In [64]:
import torch.optim as optim
from sklearn.metrics import accuracy_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    all_preds = []
    all_labels = []
    
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        all_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        print(f"\rEpoch {epoch+1}/{num_epochs} - Batch {batch_idx+1}/{len(train_loader)}", end="")

    train_acc = accuracy_score(all_labels, all_preds)
    avg_loss = running_loss / len(train_loader)

    # --- Evaluation on test set ---
    model.eval()
    test_preds = []
    test_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

    test_acc = accuracy_score(test_labels, test_preds)

    # Final print for this epoch
    print(f"\nEpoch {epoch+1}: Train Acc = {train_acc:.4f}, Test Acc = {test_acc:.4f}, Loss = {avg_loss:.4f}")


Epoch 1/10 - Batch 23810/23810
Epoch 1: Train Acc = 0.9998, Test Acc = 1.0000, Loss = 0.0011
Epoch 2/10 - Batch 23810/23810
Epoch 2: Train Acc = 1.0000, Test Acc = 1.0000, Loss = 0.0005
Epoch 3/10 - Batch 13311/23810

KeyboardInterrupt: 

###### decent

In [57]:
import torch.nn as nn
import torch.optim as optim

class FurtherSimplifiedConv1DModel(nn.Module):
    def __init__(self, input_length):
        super(FurtherSimplifiedConv1DModel, self).__init__()
        
        # Single Convolutional Layer with fewer filters
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=4, kernel_size=3, padding=1)  # Even fewer filters
        self.pool = nn.MaxPool1d(kernel_size=2)  # Max pooling
        
        # Fully connected layers with fewer neurons
        self.fc1 = nn.Linear(4 * (input_length // 2), 8)  # Smaller fully connected layer
        self.fc2 = nn.Linear(8, 2)  # Output layer for 2 classes
        
        # Dropout with a higher probability to prevent overfitting
        self.dropout = nn.Dropout(p=0.8)  # Increased dropout rate to 80%

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # Conv1D + MaxPool1D
        x = x.view(x.size(0), -1)  # Flatten after pooling
        x = torch.relu(self.fc1(x))  # Fully connected layer
        x = self.dropout(x)  # Apply dropout
        return self.fc2(x)  # Output layer

# Instantiate the model with regularization (weight decay)
model = FurtherSimplifiedConv1DModel(input_length=X.shape[1])

# Define loss and optimizer with stronger L2 regularization
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-3)  # Increased weight decay

# Loss function
criterion = nn.CrossEntropyLoss()

# Training loop (same as before)
num_epochs = 10
print_interval = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Accuracy tracking
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # Print progress every `print_interval` batches
        if (i + 1) % print_interval == 0:
            sys.stdout.write(f"\rEpoch {epoch+1}/{num_epochs} - Batch {i+1}/{len(train_loader)}")
            sys.stdout.flush()

    # Epoch summary
    epoch_accuracy = 100 * correct / total
    print(f" --> Loss: {running_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # Classification report after each epoch
    print(classification_report(all_labels, all_preds, target_names=["Benign", "Attack"], digits=4))


Epoch 1/10 - Batch 59500/59523 --> Loss: 11430.6578, Accuracy: 92.21%
              precision    recall  f1-score   support

      Benign     0.9160    0.9999    0.9561   1615643
      Attack     0.9985    0.4873    0.6550    289083

    accuracy                         0.9221   1904726
   macro avg     0.9572    0.7436    0.8055   1904726
weighted avg     0.9285    0.9221    0.9104   1904726

Epoch 2/10 - Batch 21000/59523

KeyboardInterrupt: 

##### High Accuracy

In [55]:
import torch.nn as nn
import torch.optim as optim

class SimpleConv1DModelWithDropout(nn.Module):
    def __init__(self, input_length):
        super(SimpleConv1DModelWithDropout, self).__init__()
        
        # Convolutional Layer with Dropout
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=8, kernel_size=3, padding=1)  # Even fewer filters
        self.pool = nn.MaxPool1d(kernel_size=2)  # Max pooling
        
        # Fully connected layers
        self.fc1 = nn.Linear(8 * (input_length // 2), 16)  # Reduce FC size
        self.fc2 = nn.Linear(16, 2)  # Output layer for 2 classes
        
        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(p=0.5)  # Dropout rate of 50%

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # Conv1D + MaxPool1D
        x = x.view(x.size(0), -1)  # Flatten after pooling
        x = torch.relu(self.fc1(x))  # Fully connected layer
        x = self.dropout(x)  # Apply dropout
        return self.fc2(x)  # Output layer


In [56]:
# Instantiate the model with regularization (weight decay)
model = SimpleConv1DModelWithDropout(input_length=X.shape[1])

# Define loss and optimizer with L2 regularization (weight decay)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  # Small weight decay to prevent overfitting

# Loss function
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 10
print_interval = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Accuracy tracking
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # Print progress every `print_interval` batches
        if (i + 1) % print_interval == 0:
            sys.stdout.write(f"\rEpoch {epoch+1}/{num_epochs} - Batch {i+1}/{len(train_loader)}")
            sys.stdout.flush()

    # Epoch summary
    epoch_accuracy = 100 * correct / total
    print(f" --> Loss: {running_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # Classification report after each epoch
    print(classification_report(all_labels, all_preds, target_names=["Benign", "Attack"], digits=4))


Epoch 1/10 - Batch 59500/59523 --> Loss: 292.9487, Accuracy: 99.81%
              precision    recall  f1-score   support

      Benign     0.9978    0.9999    0.9989   1615643
      Attack     0.9996    0.9879    0.9937    289083

    accuracy                         0.9981   1904726
   macro avg     0.9987    0.9939    0.9963   1904726
weighted avg     0.9981    0.9981    0.9981   1904726

Epoch 2/10 - Batch 59500/59523 --> Loss: 139.1888, Accuracy: 99.88%
              precision    recall  f1-score   support

      Benign     0.9986    1.0000    0.9993   1615643
      Attack     0.9999    0.9922    0.9960    289083

    accuracy                         0.9988   1904726
   macro avg     0.9993    0.9961    0.9977   1904726
weighted avg     0.9988    0.9988    0.9988   1904726

Epoch 3/10 - Batch 59500/59523 --> Loss: 130.3780, Accuracy: 99.88%
              precision    recall  f1-score   support

      Benign     0.9986    1.0000    0.9993   1615643
      Attack     0.9999    0.9922

KeyboardInterrupt: 

In [49]:
import torch.nn as nn

class SimpleConv1DModel(nn.Module):
    def __init__(self, input_length):
        super(SimpleConv1DModel, self).__init__()
        
        # Single Convolutional Layer
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)  # smaller filters
        self.pool = nn.MaxPool1d(kernel_size=2)  # Keep max pooling, but less aggressive
        
        # Fully connected layers
        self.fc1 = nn.Linear(16 * (input_length // 2), 32)  # Reduced number of neurons
        self.fc2 = nn.Linear(32, 2)  # Output layer for 2 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # Conv1D + MaxPool1D
        x = x.view(x.size(0), -1)  # Flatten after pooling
        x = torch.relu(self.fc1(x))  # Fully connected
        return self.fc2(x)  # Output layer


In [54]:
# import sys
# from sklearn.metrics import classification_report

# model = SimpleConv1DModel(input_length=X.shape[1])

# num_epochs = 10
# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
#     correct = 0
#     total = 0
#     all_preds = []
#     all_labels = []

#     for i, (inputs, labels) in enumerate(train_loader):
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#         # Accuracy tracking
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
#         all_preds.extend(predicted.cpu().numpy())
#         all_labels.extend(labels.cpu().numpy())

#         # Print same line update
#         sys.stdout.write(f"\rEpoch {epoch+1}/{num_epochs} - Batch {i+1}/{len(train_loader)}")
#         sys.stdout.flush()

#     # Epoch summary
#     epoch_accuracy = 100 * correct / total
#     print(f" --> Loss: {running_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

#     # Classification report
#     print(classification_report(all_labels, all_preds, target_names=["Benign", "Attack"], digits=4))

# Instantiate the simpler model
model = SimpleConv1DModel(input_length=X.shape[1])

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
print_interval = 70  # Print every 100 batches
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Accuracy tracking
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # Print progress every `print_interval` batches
        if (i + 1) % print_interval == 0:
            sys.stdout.write(f"\rEpoch {epoch+1}/{num_epochs} - Batch {i+1}/{len(train_loader)}")
            sys.stdout.flush()

    # Epoch summary
    epoch_accuracy = 100 * correct / total
    print(f" --> Loss: {running_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # Classification report after each epoch
    print(classification_report(all_labels, all_preds, target_names=["Benign", "Attack"], digits=4))


Epoch 1/10 - Batch 59500/59523 --> Loss: 36.5066, Accuracy: 99.99%
              precision    recall  f1-score   support

      Benign     0.9999    1.0000    0.9999   1615643
      Attack     1.0000    0.9994    0.9997    289083

    accuracy                         0.9999   1904726
   macro avg     0.9999    0.9997    0.9998   1904726
weighted avg     0.9999    0.9999    0.9999   1904726

Epoch 2/10 - Batch 59500/59523 --> Loss: 9.5794, Accuracy: 100.00%
              precision    recall  f1-score   support

      Benign     1.0000    1.0000    1.0000   1615643
      Attack     1.0000    1.0000    1.0000    289083

    accuracy                         1.0000   1904726
   macro avg     1.0000    1.0000    1.0000   1904726
weighted avg     1.0000    1.0000    1.0000   1904726

Epoch 3/10 - Batch 59500/59523 --> Loss: 9.7528, Accuracy: 100.00%
              precision    recall  f1-score   support

      Benign     1.0000    1.0000    1.0000   1615643
      Attack     1.0000    1.0000   

KeyboardInterrupt: 

In [ ]:
model.eval()  # Set the model to evaluation mode

# To evaluate accuracy or other metrics, loop over the validation data similar to the training loop.
